In [ ]:
# Import modules
from pyspark.ml import Pipeline
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import VectorAssembler, StringIndexer, IndexToString, SQLTransformer
from pyspark.ml.classification import *
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType, BooleanType, DoubleType

In [ ]:
# Define input path and constants
inputData = "/data/students/bigdata-01QYD/Lab9_DBD/Reviews.csv"

In [ ]:
# Load the data
# Create a DataFrame from Reviews.csv
reviews = spark.read.load(inputData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [ ]:
reviews.show()

In [ ]:
# Select only the records with HelpfulnessDenominator>0 (i.e., rated reviews)
...

In [ ]:
# Create and compute the value of Column label for the selected rated reviews
...

In [ ]:
# Split the dataframe with Column label in training and test set
(reviews_train, reviews_test) = ....

In [ ]:
# Create/Define the preprocessing steps and the classification algorithm you want to use 
# and the content of the pipeline that is used to train the model on reviews_train and apply it on reviews_test
# Implement a first solution with one single values in features: text length
....

In [ ]:
# Fit/Train the model
model = pipeline.fit(reviews_train)

In [ ]:
# Apply the model on the test set
predictions = model.transform(reviews_test).cache()

In [ ]:
# Compute statistics
# Accuracy, F1, weighted recall, weighted precision
evaluatorAcc = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "accuracy")
evaluatorF1 = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "f1")
evaluatorRecall = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "weightedRecall")
evaluatorPrecision = MulticlassClassificationEvaluator(labelCol="label" , predictionCol= "prediction", metricName = "weightedPrecision")

print("Accuracy:", evaluatorAcc.evaluate(predictions))
print("F1:", evaluatorF1.evaluate(predictions))
print("Weighted Recall:", evaluatorRecall.evaluate(predictions))
print("Weighted Precision:", evaluatorPrecision.evaluate(predictions))

In [ ]:
#  Compute the confusion matrix
#                     Predicted  
#  Actual       Useful   Useless
#  Useful          A        B
#  Useless          C        D

A = predictions.filter("prediction=1 and label=1").count()
B = predictions.filter("prediction=0 and label=1").count()
C = predictions.filter("prediction=1 and label=0").count()
D = predictions.filter("prediction=0 and label=0").count()

print("                       Predicted")
print("  Actual \t Useful\tUseless")
print("  Useful \t "+str(A)+ "\t\t"+str(B))
print("  Useless \t "+str(C)+ "\t\t"+str(D))

In [ ]:
# Precision and recall for the two classes
# Useful
if A+C==0:
    print("Precision(Useful): undefined")
else:
    print("Precision(Useful):"+str(A/(A+C)))
    
    
print("Recall(Useful):"+str(A/(A+B)))

# Useless 
if B+D==0:
    print("Precision(Useless): undefined")
else:
    print("Precision(Useless):"+str(D/(B+D)))
    
print("Recall(Useless):"+str(D/(C+D)))